In [6]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ["TAVILY_API_KEY"] = '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'

In [1]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama

embeddings = OllamaEmbeddings(model="llama3")
model = Ollama(model="llama3")
run_local = "Yes"
CHROMA_PATH = "Chroma_crag"

# data

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

# Load
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(url)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)
# Embed and index
if run_local == "Yes":
    embedding = embeddings
else:
    # todo
    print()

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    persist_directory=CHROMA_PATH,
    embedding=embedding,
)
retriever = vectorstore.as_retriever()

# LLM

In [11]:
### Retrieval Grader
### check liên quan giữa câu hỏi và doc

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# LLM
if run_local == "Yes":
    llm = model
else:
    print("")
    
# prompt = PromptTemplate(
#     template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
#     Here is the retrieved document: \n\n {document} \n\n
#     Here is the user question: {question} \n
#     If the document contains keywords related to the user question, grade it as relevant. \n
#     It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
#     Extract information from the following phrase.\nFormatting Instructions: {format_instructions}
#     return format dict with key: score, value: yes or no
#     """,
#     input_variables=["question", "document"],
# )

# class Score(BaseModel):
#         score: str = Field(description="yes or no")
       
# parser = JsonOutputParser(pydantic_object=Score)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

OutputParserException: Invalid json output: Here is the binary score:

{"score": "yes"}

In [ ]:
print("v")

In [12]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
if run_local == "Yes":
    llm = model
else:
    print()


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

I'm an assistant for question-answering tasks, and I'll do my best to provide a helpful response based on the provided text.

Please go ahead and ask your question about LLM-powered autonomous agents. You can ask about planning, memory, tool use, or any other aspect of this concept.


In [13]:
### Question Re-writer

# LLM
if run_local == "Yes":
    llm = model
else:
    print()

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n {question}. Improved question with no preamble: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'A great challenge!\n\nThe initial question "agent memory" is quite broad and open-ended, which makes it challenging for a vector store to retrieve relevant results efficiently.\n\nHere\'s an optimized version of the question:\n\n`What memories do agents retain?`\n\nThis improved question:\n\n1. Uses specific language: By asking about "memories", I\'m using more concrete terminology that can be easily matched with vector representations in a knowledge graph.\n2. Specifies the entity: Identifying the agent as the holder of these memories makes it easier to retrieve relevant information related to agents\' cognitive processes.\n3. Focuses on a specific aspect: By asking about what memories agents retain, I\'m narrowing down the scope and making it more likely that vector store retrieval will return relevant results.\n\nThese changes enable a vector store to efficiently search for answers related to agents\' memory retention capabilities, rather than returning irrelevant or unrelated info

# Search

In [14]:

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)

# Graph state

In [ ]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

In [ ]:
from langchain.schema import Document


def retrieve(state):
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Returns:
        state (dict): New key generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Kiểm tra tài liệu truy suất có liên quan câu hỏi k

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content, "format_instructions": parser.get_format_instructions()}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


def transform_query(state):
    """
    Cải thiện câu hỏi
    
    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}


### Edges


def decide_to_generate(state):
    """
    Quyết định tạo câu trả lời hay, tạo/cải thiện lại câu hỏi

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

# Build graph

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search_node", web_search)  # web search

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

# example

In [73]:
from pprint import pprint

# Run
inputs = {"question": "What are the types of agent memory?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---


OutputParserException: Invalid json output: After reviewing the document, I found that it mentions agent memory in the context of autonomous agents. Specifically, [8] Shinn & Labash mention "dynamic memory" and "self-reflection", which seem to be related to types of agent memory.

Based on this observation, I would grade the document as relevant, so my output is:

{"score": "yes"}

In [75]:

from pprint import pprint

# Run
inputs = {"question": "How does the AlphaCodium paper work?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---


OutputParserException: Invalid json output: After reviewing the retrieved document and the user question, I found some relevant keywords. Here's my assessment:

The document doesn't contain any direct mention of AlphaCodium, but it does discuss various topics related to large language models (LLMs), tool-augmented LLMs, and autonomous agents.

Here are the relevant keywords I extracted from the phrase "How does the AlphaCodium paper work?":

* AlphaCodium: None
* Paper: None
* Work: None
* Large language models: None
* Tool-augmented: None
* Autonomous: Yes

Given these results, I'll grade the document's relevance as:

{"score": "no"}

The document doesn't seem to be directly related to the user question about AlphaCodium. However, it does touch on topics that might be of interest if the user is exploring larger themes in LLMs and autonomous agents.

In [74]:
print("g")

g
